In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [ ]:
df = pd.read_excel(r"D:\客戶\Mt System\POS_MT-SYSTEM\2020POS Inpaq總表.xls", sheet_name="POS Inpaq")

In [ ]:
df = df[['Country', 'Distributor', 'Customer Name', 'Application',
       'Sell Date(text)', 'Product Name',
       'Q.ty', 'Sell unit \nprice USD', 'Total Sell, USD']]

In [ ]:
df.columns = ['Country', 'Distributor', 'Customer Name', 'Application',
       'Sell Date', 'Product Name', 'Q.ty', 'Sell unit price USD',
       'Total Sell, USD']

In [ ]:
df["Sell Date"] = pd.to_datetime(df["Sell Date"], dayfirst=True)

In [ ]:
plot = df.groupby([pd.Grouper(key="Sell Date", freq="M"), "Customer Name"])[["Total Sell, USD"]].sum().reset_index()

In [ ]:
x = df["Product Name"].str.split("[", expand=True)[0]

In [ ]:
df = pd.concat([df, x], axis=1)

In [ ]:
df = df.drop("Product Name", axis=1)

In [ ]:
df.columns = [            'Country',         'Distributor',       'Customer Name',
               'Application',           'Sell Date',                'Q.ty',
       'Sell unit price USD',     'Total Sell, USD',                   'Product Name']

In [ ]:
df["Slice"] = df["Product Name"].str[0:3]

In [ ]:
df["Product Name"].unique()

In [ ]:
pro_list = df["Product Name"].str[0:3].unique().tolist()

In [ ]:
pro_value = ['Dipole',
 'Patch',
 'Wireless',
 'Chip',
 'AAF',
 'Chip',
 'Chip',
 'Wireless',
 'AAF',
 'Chip',
 'Components',
 'Patch',
 'LDS/LCT',
 'Chip',
 'Chip',
 'Components',
 'AAF',
 'AAF']

In [ ]:
pro_dict = dict(zip(pro_list, pro_value))

In [ ]:
df["Type"] = df["Slice"].map(pro_dict)

In [ ]:
df = df.drop("Slice", axis=1)

In [ ]:
df["Month"] = df["Sell Date"].dt.month_name()
df["Year"] = df["Sell Date"].dt.year

In [ ]:
df.columns

In [ ]:
ana_tree = px.treemap(df, path=["Year", "Type", "Month", "Customer Name"], values="Total Sell, USD", color="Application")

## 繪製直方圖和折線圖

In [ ]:
import plotly.express as px

In [ ]:
his_sales = px.histogram(plot, x="Sell Date", y="Total Sell, USD", title="2020-2021 sales", labels={"Total Sell, USD": "Revenue", "Sell Date": "Month"})
his_sales.show()

In [ ]:
line_plot = df.groupby([pd.Grouper(key="Sell Date", freq="M")])[["Total Sell, USD"]].sum().reset_index()

In [ ]:
line_sales = px.line(line_plot, x="Sell Date", y="Total Sell, USD")
line_sales.show()

## 繪製表格和圓餅圖

In [ ]:
stat_df = df.groupby("Customer Name")[["Total Sell, USD"]].sum().reset_index()

In [ ]:
stat_df["Percentage"] = stat_df["Total Sell, USD"] / stat_df["Total Sell, USD"].sum()

In [ ]:
stat_df = stat_df[stat_df["Total Sell, USD"] != 0]

In [ ]:
stat_df = stat_df.sort_values(by="Total Sell, USD", ascending=False)

In [ ]:
stat_df = stat_df[stat_df["Percentage"] > 0.01]

In [ ]:
stat_df["Percentage"] = stat_df["Percentage"] / stat_df["Percentage"].sum()

In [ ]:
stat_df["Accum_percentage"] = stat_df["Percentage"].cumsum()

In [ ]:
style_df = stat_df.style.format({"Total Sell, USD": "${0:,.0f}", "Percentage": "{:.2%}", "Accum_percentage": "{:.2%}"}).hide_index()

In [ ]:
style_df

In [ ]:
pie_df = style_df.data

In [ ]:
pie_chart = px.pie(pie_df, values="Total Sell, USD", names="Customer Name")

In [ ]:
with open('POS_analysis.html', 'a') as f:
    f.write(style_df.render()) # Styler之後要用Render
    f.write(ana_tree.to_html(full_html=False, include_plotlyjs="cdn"))
    f.write(his_sales.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(line_sales.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(pie_chart.to_html(full_html=False, include_plotlyjs='cdn'))
    f.close()